In [27]:
from commonroad.common.file_reader import CommonRoadFileReader
import os
import matplotlib.pyplot as plt
from IPython import display
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object
import numpy as np
import math
from commonroad.geometry.shape import Circle
from commonroad.scenario.obstacle import StaticObstacle, ObstacleType
from commonroad.scenario.trajectory import State
from commonroad_dc.collision.collision_detection.pycrcc_collision_dispatch import create_collision_checker, \
    create_collision_object
from commonroad_dc.boundary import boundary
import commonroad_dc.pycrcc as pycrcc
from commonroad_dc.collision.trajectory_queries import trajectory_queries
 
#insert scenario to evaluate
scenarios_path = ''
scenario_id = ''
scenario, planning_problem_set = CommonRoadFileReader(scenarios_path + scenario_id).open()

visualize scenario for desired amount of time steps
for i in range(0, 118):

    # uncomment to clear previous graph
    display.clear_output(wait=True)
    
    plt.figure(figsize=(20, 10))
    draw_object(scenario, draw_params={'time_begin': i})
    draw_object(planning_problem_set)
    axis = plt.gca()
    axis.set_aspect("equal")
    plt.show()


## Longitudinal Distance

The functions below determine if the vehicle is keeping a safe longitudinal distance from other vehicles. The safe distance parameter can be changed depending on what is deemed a safe distance. One option is the 3 second rule, that any vehicle in front of the ego should be at least 3 seconds away. 

In [28]:
def create_ego_tvo_from_i(scenario, i):
    
    ego = scenario.obstacle_by_id(-1)
    car_length = ego.obstacle_shape.length
    car_width = ego.obstacle_shape.width
    state_list = ego.prediction.trajectory.state_list
    
    tvo=pycrcc.TimeVariantCollisionObject(i)

    for j in range(i, len(state_list)):
        
        #change this parameter
        safe_dist = 2
        current_pos = state_list[j].position
        orientation = state_list[j].orientation
        
        x = current_pos[0]
        y = current_pos[1]
        
        length = (car_length/2) + safe_dist
        
        #note that the collision obstacle created needs a half length input
        tvo.append_obstacle(pycrcc.RectOBB(length,car_width/2,orientation,x,y))
        
    return tvo


def create_tvo(obs, i):
    length = obs.obstacle_shape.length
    width = obs.obstacle_shape.width
    state_list = obs.prediction.trajectory.state_list
    tvo = pycrcc.TimeVariantCollisionObject(i)
    
    for j in range(i, len(state_list)):
        pos = state_list[j].position
        orientation = state_list[j].orientation
        
        x = pos[0]
        y = pos[1]
        
        tvo.append_obstacle(pycrcc.RectOBB(length/2,width/2,orientation,x,y))
    return tvo


def recurse(scenario, obs, i, count) -> int:
    
    co = create_ego_tvo_from_i(scenario, i)
    preprocessed_trajectory, err=trajectory_queries.trajectory_preprocess_obb_sum(co)
    if(err):
        raise Exception("trajectory preprocessing error")
    
    ret=trajectory_queries.trajectories_collision_dynamic_obstacles([preprocessed_trajectory], obs)
    if ret[0] == -1:
        print('Done')
        return 0
    else:
        print('Time step of violation %s: %s' % (count, ret[0]))
        return recurse(scenario, obs, ret[0]+1, count+1)


def safe_long_dist_violations(scenario):
    dynamic_obstacles = []
    for obs in scenario.dynamic_obstacles:
        if obs.obstacle_id != -1:
            tvo = create_tvo(obs, 0)
            dynamic_obstacles.append(tvo)
    recurse(scenario, dynamic_obstacles, 0, 1)

#safe_long_dist_violations(scenario)

## Lane Tracking
Lane tracking is measured by creating a collision object out of each lanelet in the network. Then, each time step during which the ego is not entirely enclosed within a lanelet is tallied. 

In [29]:
def recurse(ego, road_boundary, i, count) -> int:
    co = create_tvo(ego, i)
    preprocessed_trajectory, err=trajectory_queries.trajectory_preprocess_obb_sum(co)
    if(err):
        raise Exception("trajectory preprocessing error")

    ret=trajectory_queries.trajectories_enclosure_polygons_static([preprocessed_trajectory], road_boundary, method='grid', num_cells=32, enable_verification=False)
    if ret[0] == -1:
        print('Done')
        return 0
    else:
        print('Time step of incomplete enclosure in lanelet %s: %s' % (count, ret[0]))
        return recurse(ego, road_boundary, ret[0]+1, count+1)

def lane_tracking_violation(scenario):
    
    ego = scenario.obstacle_by_id(-1)
    road_boundary = boundary.create_road_polygons(scenario, method='lane_polygons', buffer=1,resample=1, triangulate=False)
    
    recurse(ego, road_boundary, 0, 1)
    
    
    
#lane_tracking_violation(scenario)

## Relative Speed
Here, the differential between the ego's current velocity and the average velocity of all other vehicles is compared. The function returns an array of the differential at each time step (return type can be modified for different use cases)

In [13]:
def relative_speed(scenario):    
    avg_v = []
    differential_v = []
    
    ego = scenario.obstacle_by_id(-1)
    state_list = ego.prediction.trajectory.state_list
    
    count = len(scenario.dynamic_obstacles)
    
    for i in range(0, len(state_list)):
        total_v = 0
        for obs in scenario.dynamic_obstacles:
            if obs.obstacle_id != -1:
                obs_v = obs.prediction.trajectory.state_list[i].velocity
                total_v += obs_v
        avg_v.append(total_v/count)
        ego_v = state_list[i].velocity
        diff_v = ego_v - (total_v/count)
        differential_v.append(diff_v)
    return differential_v